본 프로젝트는 Meta의 LLama 3.3 모델을 활용하여 구현되었습니다. LLama 모델은 [[해당 라이선스 링크](https://github.com/meta-llama/llama-models/blob/main/models/llama3_3/LICENSE)]를 따릅니다.


In [27]:
from datasets import load_dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments, pipeline, AutoModelForSeq2SeqLM
import os
import torch
from trl import SFTTrainer
from huggingface_hub import notebook_login

In [2]:
notebook_login()

In [3]:
#네이버 뉴스 요약 데이터셋 로드
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

In [4]:
# 'document'와 'summary' 컬럼만 선택하기
train_data = dataset['train'].map(lambda x: {'input': x['document'], 'output': x['summary']})
validation_data = dataset['validation'].map(lambda x: {'input': x['document'], 'output': x['summary']})
test_data = dataset['test'].map(lambda x: {'input': x['document'], 'output': x['summary']})

In [5]:
#!huggingface-cli login

In [6]:
# Hugging Face 캐시 경로 변경
#os.environ["HF_HOME"] = "D:/huggingface_cache" # 공간없을 때 사용용

#BASE_MODEL = "meta-llama/Llama-3.3-70B-Instruct" # 학습 환경 최소GPU 48G 필요
#BASE_MODEL = "meta-llama/Llama-3.2-1B-Instruct-SpinQuant_INT4_EO8"
BASE_MODEL = "meta-llama/Llama-3.2-1B-Instruct"

# LoRA 설정 : 양자화된 모델에서 Adaptor를 붙여서 학습할 파라미터만 따로 구성함
lora_config = LoraConfig(
    r=8,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

# 4bit 양자화 설정 - QLoRA로 해야 함
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", #nf4
    bnb_4bit_use_double_quant=True, #True
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True
)

# Protobuf 라이브러리 설치 후 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# EOS 토큰을 패딩 토큰으로 설정 (필요 시)
tokenizer.pad_token = tokenizer.eos_token

#모델 로드 (양자화 )
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL,
                                              use_auth_token = "{huggingFaceToken}", # or login
                                              quantization_config=bnb_config,
                                              device_map="auto",  # Automatically assign devices
                                                #llm_int8_enable_fp32_cpu_offload=True  # CPU offload if necessary
                                             )

c:\anaconda3\envs\LLama\lib\site-packages\transformers\models\auto\auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [7]:
# Protobuf 라이브러리 설치 후 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# EOS 토큰을 패딩 토큰으로 설정 (필요 시)
tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = 'right'

In [8]:
def generate_prompts(example):
    prompt_list = []
    for i in range(len(example['document'])):
        prompt_list.append(
f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>다음 글을 요약해주세요:
{example['document'][i]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{example['summary'][i]}<|eot_id|>"""
        )
    return prompt_list

In [9]:
total_steps = len(dataset) * 1  # 예시로 1 epoch 기준, 필요한 만큼 조정
warmup_steps = int(0.03 * total_steps)  # 전체 단계 수의 3% 계산

In [10]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=validation_data,
    #max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 1,
        max_steps=300,
        per_device_train_batch_size=4, # GPU당 4개 배치
        gradient_accumulation_steps=1, # gradient 반영을 1개 step 마다
        optim="paged_adamw_8bit",
        warmup_steps=warmup_steps,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=4,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,   # LoRA 설정값
    formatting_func=generate_prompts,   # 프롬프트 템플릿 함수 해당값은 고정인것 같음 더 학습 필요
)

trainer.train()

Step,Training Loss
4,3.500500
8,3.213400
12,3.154700
16,3.014800
20,3.031900
24,2.857400
28,2.956700
32,2.794300
36,2.943300
40,2.919100


TrainOutput(global_step=300, training_loss=2.743792012532552, metrics={'train_runtime': 5507.2708, 'train_samples_per_second': 0.218, 'train_steps_per_second': 0.054, 'total_flos': 6635814499418112.0, 'train_loss': 2.743792012532552, 'epoch': 0.05406379527842855})

In [14]:
# 모델 저장
trainer.model.save_pretrained("LLama_fine_tuned_model")
tokenizer.save_pretrained("LLama_fine_tuned_model")

('LLama_fine_tuned_model\\tokenizer_config.json',
 'LLama_fine_tuned_model\\special_tokens_map.json',
 'LLama_fine_tuned_model\\tokenizer.json')

간단 Result 번역 포함은 다른 파일에서 진행

In [17]:
# 출처 조선일보 탑픽
document = f"""
텍스트 입력
"""

pipe = pipeline("text-generation", 
                model=model, 
                tokenizer=tokenizer, 
                max_new_tokens=256,
                device_map="auto",
                )

messages = [
    {"role": "user", "content": f"""
    Below is a Korean document. Your task is to:
    1. Translate it into English.
    2. Reformat it into a concise and AI-friendly English prompt.

    Document:
    {document}
    """},
]

prompt = pipe.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.4,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True,
    eos_token_id = [ # eos_token_id를 지정하지 않으면 생성 토큰 반복
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
)

Device set to use cuda:0


이날 취재진과 만나 공수처와 경찰은 경호처 측에 ‘안전하고 평화적인 영장 집행을 위한 협조’를 요청했으나, 경호처 측이 이에 대해 명시적인 답변을 내놓진다는 것이 입장이다. 공수처 관계자는 ‘영장 집행을 위해 협조해달라는 정도의 의견이 오고 갔다’며 ‘영장 집행을 위해 협조해달라는 정도의 의견이 오고 갔다’라고 밝혔다. 이날 취재진과 만나 ‘안전하고 평화적인 영장 집행을 위한 협조’를 요청했으나, 경호처 측이 이에 대해 명시적인 답변을 내놓진다는 것이 입장이다. 공수처와 경찰은 경호처 측의 회신을 기다리고 있는데, 정해진 답변 시한은 없다는 것이 이 관계자 설명이다. 공수처 관계자는 ‘영장 집행을 위해 협조해달라는 정도의 의견이 오고 갔다’며 ‘영장 집행을 위해 협조해달라는 정도의 의견이 오고 갔다’라고 밝혔다. 이
